In [1]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.RollRig as RollRig

from matplotlib.lines import Line2D

me.mechanics_printing()

In [2]:
half = sym.Rational(1,2)

p = RollRig.base_params(1)

In [3]:
# Generate Referecne Frame
wing_frame = mf.HomogenousTransform().R_x(p.q[0])#.R_y(p.alpha_r)
rhs_fwt_frame = wing_frame.Translate(0,p.s_w*half,0)  # RHS Fwt Frame
lhs_fwt_frame = wing_frame.Translate(0,-p.s_w*half,0)  # RHS Fwt Frame

#Generate Mass Matrices
M_w = ele.MassMatrix(p.m_w)
I_w = ele.MassMatrix(0,I_xx = p.I_xx_w)

# Generate Rigid Elements
Wing_point_mass = ele.RigidElement(wing_frame.Translate(0,p.eta_0,p.eta_1),M_w,gravityPotential=True)
wing_inertia = ele.RigidElement(wing_frame,I_w)

In [4]:
# Main Wing Aero Forces 
wing_AeroForces = ef.AeroForce.PerUnitSpan(p,wing_frame.Translate(0,p.y_w,0),p.a_0,
                               alphadot = 0,
                               M_thetadot = 0,
                               e = 0,
                               w_g = 0,
                               rootAlpha = 0,
                               alpha_zero = 0,
                               stall_angle = 0,
                               c = p.c,
                               c_d_max = 1,
                               linear = True).integrate((p.y_w,-p.s_w*half,p.s_w*half))

In [5]:
wing_AeroForces.Q()

⎡           3        3    ⎤
⎢-V⋅a₀⋅c⋅ρ⋅s ⋅(1 - σ) ⋅q₀̇ ⎥
⎢─────────────────────────⎥
⎣            24           ⎦

In [6]:
# Left FWT Aero Forces 
lhs_fwt_AeroForces = ef.AeroForce.PerUnitSpan(p,lhs_fwt_frame.Translate(0,-p.y_f,0),p.a_1,
                               alphadot = 0,
                               M_thetadot = 0,
                               e = 0,
                               w_g = p.w_g,
                               rootAlpha = p.alpha_2,
                               alpha_zero = 0,
                               stall_angle = 0,
                               c = p.c,
                               c_d_max = p.c_d_max,
                               linear = False)#.integrate((p.y_f,0,p.s_f))

In [7]:
# Left FWT Aero Forces 
rhs_fwt_AeroForces = ef.AeroForce.PerUnitSpan(p,rhs_fwt_frame.Translate(0,p.y_f,0),p.a_1,
                               alphadot = 0,
                               M_thetadot = 0,
                               e = 0,
                               w_g = p.w_g,
                               rootAlpha = p.alpha_1,
                               alpha_zero = 0,
                               stall_angle = 0,
                               c = p.c,
                               c_d_max = p.c_d_max,
                               linear = True)#.integrate((p.y_f,0,p.s_f))

In [8]:
# split FTW's into segments
forces = []
segments = 5
for i in range(segments):
    seg_width = p.s_f/segments
    yi = seg_width/2 + i*seg_width
    forces.append(lhs_fwt_AeroForces.subs({p.y_f:yi,p.a_1:p.a_1/segments*(4-i+0.5)})*seg_width)
    forces.append(rhs_fwt_AeroForces.subs({p.y_f:yi})*seg_width)
Q = sym.Matrix([0]*p.qs)
for f in forces:
    Q += f.Q()
fwt_AeroForces = ef.ExternalForce(Q)



In [9]:
CompositeForce = ef.CompositeForce([fwt_AeroForces,wing_AeroForces])
# create instance of the model
sm = mf.SymbolicModel.FromElementsAndForces(p,[Wing_point_mass,wing_inertia],ExtForces = CompositeForce)

In [10]:
sm.to_file('RollRigModel-Fixed.py')